# 🚀 머신러닝 실습 : 고객 구매 데이터로 성별 예측 모델링 (분류 문제)

* 주어진 데이터는 백화점 고객의 1년 간 구매 데이터입니다.
* 고객 3,500명에 대한 학습용 데이터(y.csv, X.csv)를 이용하여 성별예측 모형을 만들어보세요.
* 모델의 성능은 자유롭게 측정해봅니다!

## [실습 프로세스]
1. 데이터 불러오기  
2. 데이터 탐색
3. 데이터 전처리  
4. 학습/테스트 데이터 분리  
5. 모델 선택 및 학습  
6. 예측 및 평가  


<br/>

---

<br/>
<br/>

# 0. 라이브러리 불러오기

* 라이브러리를 가져와서 과정을 준비합니다

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder

<br/>

---

<br/>
<br/>

# 1. 데이터 불러오기
* 데이터를 가져와서 과정을 준비합시다.
- 인코딩 방식은 'euc-kr' 을 활용하세요.
- 데이터 출처 : 한국데이터산업진흥원 빅데이터분석기사 실기 공개 예시 문항

- 독립 변수 데이터셋 : ./data/X.csv
- 종속 변수 데이터셋 : ./data/y.csv


데이터 파일을 불러옵니다. 보통 CSV 파일을 pandas로 읽어옵니다.

In [ ]:
# import os
# # 노트북 파일이 있는 폴더로 이동 (예시)
# os.chdir(r'C:\githome\hipython_rep')

# # 변경 후 확인
# print("변경 후:", os.getcwd())

변경 후: c:\githome\hipython_rep


In [21]:
# 1. 데이터 불러오기
X = pd.read_csv('./data/X.csv', encoding='euc-kr')
y = pd.read_csv('./data/y.csv', encoding='euc-kr')

In [3]:
X.head()

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17
1,1,2136000,2136000,300000.0,스포츠,잠실점,2,1.500000,0.000000,1
2,2,3197000,1639000,NaN,남성 캐주얼,관악점,2,2.000000,0.000000,1
3,3,16077620,4935000,NaN,기타,광주점,18,2.444444,0.318182,16
4,4,29050000,24000000,NaN,보석,본 점,2,1.500000,0.000000,85


In [9]:
y.head(10)

,cust_id,gender
0,0,0
1,1,0
2,2,1
3,3,1
4,4,0
5,5,0
6,6,0
7,7,0
8,8,0
9,9,1


<br/>

---

<br/>
<br/>

# 2. 데이터 탐색하기
* 데이터를 이해할 수 있도록 탐색과정을 수행해봅시다.


데이터의 상위 몇 개 행을 출력하여 전체 구조를 미리 확인합니다.


데이터의 요약 정보나 통계 정보를 출력해 변수들의 유형과 분포를 확인합니다.


데이터의 요약 정보나 통계 정보를 출력해 변수들의 유형과 분포를 확인합니다.

In [13]:
X.info()

<class 'pandas.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  3500 non-null   int64  
 1   총구매액     3500 non-null   int64  
 2   최대구매액    3500 non-null   int64  
 3   환불금액     1205 non-null   float64
 4   주구매상품    3500 non-null   str    
 5   주구매지점    3500 non-null   str    
 6   내점일수     3500 non-null   int64  
 7   내점당구매건수  3500 non-null   float64
 8   주말방문비율   3500 non-null   float64
 9   구매주기     3500 non-null   int64  
dtypes: float64(3), int64(5), str(2)
memory usage: 273.6 KB


In [15]:
X.columns

Index(['cust_id', '총구매액', '최대구매액', '환불금액', '주구매상품', '주구매지점', '내점일수', '내점당구매건수',
       '주말방문비율', '구매주기'],
      dtype='str')

In [11]:
X['환불금액']

0       6860000.0
1        300000.0
2             NaN
3             NaN
4             NaN
          ...    
3495          NaN
3496    6049600.0
3497          NaN
3498          NaN
3499    5973000.0
Name: 환불금액, Length: 3500, dtype: float64

<br/>

---

<br/>
<br/>

# 3. 데이터 전처리
* 전처리 과정을 통해서 머신러닝에 사용할 수 있는 형태의 데이터 준비


필요한 라이브러리를 불러옵니다.
- 인코딩 : LabelEncoder
- 데이터 표준화 : StandardScaler

* 단순히 1부터의 숫자를 부여한 'cust_id'를 수치형 변수로 받아들이면, 결과가 왜곡될 수 있으니 컬럼을 제거합니다.

- 데이터에 결측치가 있는지 확인해보세요


- 결측치에 0으로 채워 넣어 모델 학습에 지장이 없도록 합니다.


문자형 범주 데이터를 숫자로 바꾸기 위한 인코딩을 수행합니다.

각 데이터에 표준화를 적용하여 데이터의 스케일(크기 차이)을 맞춰줍니다.
- 평균을 0, 표준편차를 1로 맞춰서 → 데이터가 정규 분포 형태로 변환되도록 하세요

In [22]:
# 2. 환불금액 결측치 → 0으로 대체
X['환불금액'] = X['환불금액'].fillna(0)

# 3. cust_id를 gender로 대체
#    y의 cust_id 기준으로 gender 매핑
gender_map = dict(zip(y['cust_id'], y['gender']))
X['cust_id'] = X['cust_id'].map(gender_map)
X = X.rename(columns={'cust_id': 'gender'})

In [23]:
final_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   총구매액     3500 non-null   int64  
 1   최대구매액    3500 non-null   int64  
 2   환불금액     3500 non-null   float64
 3   주구매상품    3500 non-null   str    
 4   주구매지점    3500 non-null   str    
 5   내점일수     3500 non-null   int64  
 6   내점당구매건수  3500 non-null   float64
 7   주말방문비율   3500 non-null   float64
 8   구매주기     3500 non-null   int64  
 9   gender   3500 non-null   int64  
dtypes: float64(3), int64(5), str(2)
memory usage: 273.6 KB


<br/>

---

<br/>
<br/>

# 5-1. 모델링 - LogisticRegression

* 본격적으로 모델을 선언하고 학습시킵니다.


필요한 라이브러리를 불러옵니다.

모델을 선언하여 객체화시킵니다.


모델을 학습 데이터에 맞춰 학습시킵니다.

In [ ]:
# 4. 범주형 컬럼 인코딩 (주구매상품, 주구매지점, gender)
le = LabelEncoder()
for col in ['주구매상품', '주구매지점']:
    X[col] = le.fit_transform(X[col].astype(str))

target = le.fit_transform(X['gender'].astype(str))  # gender → 숫자
X = X.drop(columns=['gender'])

In [25]:
# 5. train / test 분리 (8:2)
X_train, X_test, y_train, y_test = train_test_split(
    X, target, test_size=0.2, random_state=42, stratify=target
)

<br/>

---

<br/>
<br/>

# 6-1. 예측 성능 확인해보기 - LogisticRegression

- 학습된 모델로 테스트 데이터에 대한 예측을 수행합니다.

- 학습시킨 모델의 성능을 알아봅니다
- 각 평가지표로 모델의 성능을 수치화하여 확인합니다.
- 필요한 라이브러리를 import 하고 성능을 확인해보세요 (정확도, 정밀도, 재현율, f1, confusion_matrix)

In [26]:
# 6. 모델 정의
models = {
    'LogisticRegression'    : LogisticRegression(max_iter=1000, random_state=42),
    'DecisionTree'          : DecisionTreeClassifier(random_state=42),
    'RandomForest'          : RandomForestClassifier(random_state=42),
    'XGBoost'               : XGBClassifier(eval_metric='logloss', random_state=42),
}

# 7. 학습 및 평가
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print(f"\n{'='*45}")
    print(f"  {name}")
    print(f"{'='*45}")
    print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}")
    print(f"Accuracy  : {accuracy_score(y_test, y_pred):.4f}")
    print(f"Precision : {precision_score(y_test, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"Recall    : {recall_score(y_test, y_pred, average='weighted', zero_division=0):.4f}")


  LogisticRegression
Confusion Matrix:
[[437   0]
 [263   0]]
Accuracy  : 0.6243
Precision : 0.3897
Recall    : 0.6243

  DecisionTree
Confusion Matrix:
[[283 154]
 [150 113]]
Accuracy  : 0.5657
Precision : 0.5670
Recall    : 0.5657

  RandomForest
Confusion Matrix:
[[355  82]
 [174  89]]
Accuracy  : 0.6343
Precision : 0.6145
Recall    : 0.6343

  XGBoost
Confusion Matrix:
[[333 104]
 [150 113]]
Accuracy  : 0.6371
Precision : 0.6261
Recall    : 0.6371



<br/>

---

<br/>
<br/>

# 5-2. 모델링 - DecisionTreeClassifier

* 본격적으로 모델을 선언하고 학습시킵니다.


필요한 라이브러리를 불러옵니다.

모델을 선언하여 객체화시킵니다.

모델을 학습 데이터에 맞춰 학습시킵니다.



<br/>
<br/>

# 6-2. 예측 성능 확인해보기 - DecisionTreeClassifier

- 학습된 모델로 테스트 데이터에 대한 예측을 수행합니다.

- 학습시킨 모델의 성능을 알아봅니다
- 각 평가지표로 모델의 성능을 수치화하여 확인합니다.
- 필요한 라이브러리를 import 하고 성능을 확인해보세요 (정확도, 정밀도, 재현율, f1, confusion_matrix)


<br/>

---

<br/>
<br/>

# 5-3. 모델링 - RandomForestClassifier

* 본격적으로 모델을 선언하고 학습시킵니다.



필요한 라이브러리를 불러옵니다.

모델을 선언하여 객체화시킵니다.

모델을 학습 데이터에 맞춰 학습시킵니다.



<br/>
<br/>

# 6-3. 예측 성능 확인해보기 - RandomForestClassifier

- 학습된 모델로 테스트 데이터에 대한 예측을 수행합니다.

- 학습시킨 모델의 성능을 알아봅니다
- 각 평가지표로 모델의 성능을 수치화하여 확인합니다.
- 필요한 라이브러리를 import 하고 성능을 확인해보세요 (정확도, 정밀도, 재현율, f1, confusion_matrix)


<br/>

---

<br/>
<br/>

# 5-4. 모델링 - XGBoost

* 본격적으로 모델을 선언하고 학습시킵니다.



필요한 라이브러리를 불러옵니다.

모델을 선언하여 객체화시킵니다.

모델을 학습 데이터에 맞춰 학습시킵니다.



<br/>
<br/>

# 6-4. 예측 성능 확인해보기 - XGBoost

- 학습된 모델로 테스트 데이터에 대한 예측을 수행합니다.

- 학습시킨 모델의 성능을 알아봅니다
- 각 평가지표로 모델의 성능을 수치화하여 확인합니다.
- 필요한 라이브러리를 import 하고 성능을 확인해보세요 (정확도, 정밀도, 재현율, f1, confusion_matrix)

<br/>

---


<br/>

## 7.  위 4가지 모델의 학습 & 예측 & 평가 결과를 확인하고 최고 성능을 내는 모델을 찾아봅시다!

- 어떤 모델이 가장 성능이 좋은가요 ?